In [65]:
from Bio.PDB import Polypeptide
import gromacs

In [88]:
def parseGro(f):
    
    fp = open(f, 'r')
    pp = list(fp)
    res = ''
    atlist = []
    aas = []
    prevnum = 0
    for i in range(2, 1000):
        line = pp[i].split()
        num = int(line[0][:-3])
        aa = line[0][-3:]
        
        if not Polypeptide.is_aa(aa):
            break;
            
        shortaa = Polypeptide.three_to_one(aa)
        if (num != prevnum):
            atlist.append(i - 1)
            aas.append(aa + '_' + str(num))
            prevnum = num
            res = res + shortaa
            
    fp.close()
    return res, atlist, aas

In [67]:
def findSeqsInGro(seq, pep, cdr3_a, cdr3_b):
    pbeg = seq.find(pep) + 1
    pend = pbeg + len(pep) - 1
    abeg = seq.find(cdr3_a) + 1
    aend = abeg + len(cdr3_a) - 1
    bbeg = seq.find(cdr3_b) + 1
    bend = bbeg + len(cdr3_b) - 1
    return [pbeg, pend], [abeg, aend], [bbeg, bend]

In [68]:
def appendSeqs(path, idlist, atlist, aas):
    fl = open(path, 'a')
    groups = gromacs.cbook.get_ndx_groups(path)
    names = []
    for pair in idlist:
        for aaid in range(pair[0] - 1, pair[1]):
            name = 'r_' + str(pair[0]) + '-' + str(pair[1]) + '_' + aas[aaid]
            names.append(name)
            if any(group['name'] == name for group in groups):
                continue
            fl.write('[ ' + name + ' ]\n')
            for item in range(atlist[aaid], atlist[aaid + 1]):
                fl.write(str(item) + ' ')
            fl.write('\n')
    fl.close()
    return names

In [69]:
def appendToMDP(path, grnames):
    replaceText = 'energygrps\t='
    for name in grnames:
        replaceText = replaceText + ' ' + name
    replaceText = replaceText + '\n'
    
    fl = open(path, 'r')
    text = fl.read()
    fl.close()
    
    fl = open(path, 'r')
    s = ''
    for line in fl:
        if line.split()[0] == 'energygrps':
            s = line
            break
    fl.close()
    
    fl = open(path, 'w')
    if not s:
        fl.write(text + replaceText)
    else:
        fl.write(text.replace(s, replaceText))
    fl.close()

In [70]:
def appendToGroupsDat(path, names):
    fl = open(path, 'w')
    fl.write(str(len(names)) + '\n')
    for name in names:
        fl.write(name + '\n')

In [71]:
st, atlist, aas = parseGro('2bnr.gro')

In [87]:
#parseGro('2bnr.gro')

In [72]:
p, a, b = findSeqsInGro(st, 'SLLMWITQC', 'CAVRPTSGGSYIPTF', 'CASSYVGNTGELFF')

In [73]:
names = appendSeqs('index.ndx', [p, a, b], atlist, aas)

In [74]:
names

['r_377-385_SER_1',
 'r_377-385_LEU_2',
 'r_377-385_LEU_3',
 'r_377-385_MET_4',
 'r_377-385_TRP_5',
 'r_377-385_ILE_6',
 'r_377-385_THR_7',
 'r_377-385_GLN_8',
 'r_377-385_CYS_9',
 'r_474-488_CYS_90',
 'r_474-488_ALA_91',
 'r_474-488_VAL_92',
 'r_474-488_ARG_93',
 'r_474-488_PRO_94',
 'r_474-488_THR_95',
 'r_474-488_SER_96',
 'r_474-488_GLY_97',
 'r_474-488_GLY_98',
 'r_474-488_SER_99',
 'r_474-488_TYR_100',
 'r_474-488_ILE_101',
 'r_474-488_PRO_102',
 'r_474-488_THR_103',
 'r_474-488_PHE_104',
 'r_677-690_CYS_90',
 'r_677-690_ALA_91',
 'r_677-690_SER_92',
 'r_677-690_SER_93',
 'r_677-690_TYR_94',
 'r_677-690_VAL_95',
 'r_677-690_GLY_96',
 'r_677-690_ASN_97',
 'r_677-690_THR_98',
 'r_677-690_GLY_99',
 'r_677-690_GLU_100',
 'r_677-690_LEU_101',
 'r_677-690_PHE_102',
 'r_677-690_PHE_103']

In [75]:
appendToMDP('minim.mdp', names)

In [76]:
appendToGroupsDat('groups.dat', names)

In [77]:
import gromacs.formats
import gromacs.cbook

In [78]:
G = gromacs.cbook.IndexBuilder('2bnr.gro')


In [82]:
G._process_command('l', 'name')

('name', '/tmp/name__n9Fj4k.ndx')

In [24]:
d = {'n' : 23, 'f' : 43}

In [80]:
G.cat('index.ndx')

'index.ndx'

In [64]:
pwd

u'/home/mikhail/Projects/vdjstruct/app/src/main/python'